## Mapping Gene Expression Data to CNV Tables

### Overview
This notebook maps gene expression data to the corresponding gene-level copy number variation table through the gene name. This is done to add gene expression to the ground truth table as a feature. Gene expression data is taken from the statistics files, which are generated through preprocessing dense matrices.

### Workflow:

1. Gene-level CNV data is retrieved from its original file.
2. Gene names in the CNV table are matched (joined) to those in the corresponding stats file.
3. The expression data for genes is identified in the stats file (Sum, Mean, Variance columns) and added to the CNV table.

Each joined table is then appended to a larger table that consists of all available sample data.

Import libraries

In [37]:
import pandas as pd
import requests

Get the mapping file linking Case IDs, gene-level CNV tables, and gene expression statistics. Samples without a corresponding dense matrix are dropped because no gene expression data can be pulled from them.

In [38]:
map_df = pd.read_csv('https://raw.githubusercontent.com/LaraLim/cnv-supervised-learning/refs/heads/main/stats_mapping.csv')

# list to store indices of rows with no stats (.txt file instead of .csv)
rows_to_drop = []

for index, row in enumerate(map_df['Stats_file']):
    fname = row.split(".")[0]  # get file name prefix
    url = f'https://github.com/LaraLim/cnv-supervised-learning/tree/main/gene_statistics/{fname}.txt'
    response = requests.get(url) # GET request looking for a .txt file

    # if the file exists, add row index to list
    if response.status_code == 200:
        rows_to_drop.append(index)

# drop all recorded rows by index
map_df = map_df.drop(index=rows_to_drop).reset_index(drop=True)

Retrieve gene-level CNV data for all cases and store in a dataframe dictionary

In [39]:
# dictionary of CNV dataframes identifiable by case ID
cnv_dfs = {}

# dataframe to track CNV data across all samples
combined_cnv_df = pd.DataFrame()
combined_cnv_df['CaseID'] = ''

# first sample
filename = map_df['CNV_file'][0]
url = f'https://raw.githubusercontent.com/LaraLim/cnv-supervised-learning/refs/heads/main/gene_level_cnv/{filename}'
df = pd.read_csv(url, sep='\t')
numRows = df.shape[0]

for filename in map_df['CNV_file']:

    # get the case id
    case_id = map_df[map_df['CNV_file'] == filename]['CaseID'].values[0]
    df['CaseID'] = case_id

    url = f'https://raw.githubusercontent.com/LaraLim/cnv-supervised-learning/refs/heads/main/gene_level_cnv/{filename}'
    df = pd.read_csv(url, sep='\t')

    # check if the number of rows is the same
    if df.shape[0] != numRows:
        print(f'The number of rows is not the same: {case_id}')

    df['status'] = 'normal'
    df.loc[df['copy_number'] > 2, 'status'] = 'amplified'
    df.loc[df['copy_number'] < 2, 'status'] = 'deleted'

    # move the position of the copy number column to the last column
    copy_number = df.pop('copy_number')
    df['copy_number_target'] = copy_number

    cnv_dfs[case_id] = df # add the dataframe to the dictionary
    combined_cnv_df = pd.concat([combined_cnv_df, df], ignore_index=True) # append the dataframe to the combined one

print(f'combined_cnv_df has {combined_cnv_df.shape[0]} rows')
display(combined_cnv_df.head())

combined_cnv_df has 2182428 rows


,CaseID,gene_id,gene_name,chromosome,start,end,min_copy_number,max_copy_number,status,copy_number_target
0,NaN,ENSG00000223972.5,DDX11L1,chr1,11869.0,14409.0,4.0,4.0,amplified,4.0
1,NaN,ENSG00000227232.5,WASH7P,chr1,14404.0,29570.0,4.0,4.0,amplified,4.0
2,NaN,ENSG00000278267.1,MIR6859-1,chr1,17369.0,17436.0,4.0,4.0,amplified,4.0
3,NaN,ENSG00000243485.5,MIR1302-2HG,chr1,29554.0,31109.0,4.0,4.0,amplified,4.0
4,NaN,ENSG00000284332.1,MIR1302-2,chr1,30366.0,30503.0,4.0,4.0,amplified,4.0


Retrieve gene expression statistics (for all cases) derived from dense matrices and store in a dataframe dictionary. Statistics include the sum, mean, and variance for gene expression.

In [40]:
# dictionary of gene expression stat dataframes identifiable by case ID
ge_dfs = {}

# dataframe to track gene expression stats across all samples
combined_ge_df = pd.DataFrame()

# first sample
filename = map_df['Stats_file'][0]
url = f'https://raw.githubusercontent.com/LaraLim/cnv-supervised-learning/refs/heads/main/gene_statistics/{filename}'
df = pd.read_csv(url, delimiter=',')
numRows = df.shape[0]

for filename in map_df['Stats_file']:

    # get the case id
    case_id = map_df[map_df['Stats_file'] == filename]['CaseID'].values[0]

    url = f'https://raw.githubusercontent.com/LaraLim/cnv-supervised-learning/refs/heads/main/gene_statistics/{filename}'
    df = pd.read_csv(url, delimiter=',')

    ge_dfs[case_id] = df # add the dataframe to the dictionary
    combined_ge_df = pd.concat([combined_ge_df, df], ignore_index=True) # append the dataframe to the combined one

print(f'combined_ge_df has {combined_ge_df.shape[0]} rows')
display(combined_ge_df.head())

combined_ge_df has 1316509 rows


,Gene,Sum,Mean,Variance
0,WASH7P,0.006084,4.167978e-07,1.770619e-10
1,MIR1302-2HG,0.002205,1.510508e-07,3.707889e-11
2,OR4F5,0.000527,3.610975e-08,7.165660e-12
3,AL627309.1,0.009764,6.688604e-07,4.153622e-10
4,AL627309.6,0.000800,5.478004e-08,2.699976e-11


Merge the dataframes on `gene_name` from `df_cnv` and `Gene` from `df_ge`. Use left join to preserve all rows from `df_cnv`

In [49]:
for case_id in map_df['CaseID']:
    # Check if case ID exists in both dictionaries
    if case_id not in cnv_dfs or case_id not in ge_dfs:
        print(f"\n[WARNING] skipping case_id {case_id}; missing from one or both dictionaries\n")
        continue

    # merge only if case ID exists in both dictionaries
    joined_df = pd.merge(cnv_dfs[case_id], ge_dfs[case_id], left_on='gene_name', right_on='Gene', how='left')

    # rename columns in merged dataframe
    joined_df = joined_df.rename(columns={'Sum': 'sum_gene_expr', 'Mean': 'mean_gene_expr', 'Variance': 'variance_gene_expr'})
    joined_df = joined_df.drop(columns='Gene')  # drop redundant Gene column

    # move copy_number_target to be the end (rightmost) column
    copy_number_col = joined_df.pop('copy_number_target')
    joined_df['copy_number_target'] = copy_number_col

    # temp try catch for extra iterations
    try:
        case_id_col = joined_df.pop('CaseID')
        joined_df.insert(0, 'CaseID', case_id_col) # move CaseID to be the first (leftmost) column
    except KeyError:
        break

    display(joined_df.head())
    print("\n")

,CaseID,gene_id,gene_name,chromosome,start,end,min_copy_number,max_copy_number,status,sum_gene_expr,mean_gene_expr,variance_gene_expr,copy_number_target
0,C3L-00606-01,ENSG00000223972.5,DDX11L1,chr1,11869,14409,4.0,4.0,amplified,NaN,NaN,NaN,4.0
1,C3L-00606-01,ENSG00000227232.5,WASH7P,chr1,14404,29570,4.0,4.0,amplified,0.006084,4.167978e-07,1.770619e-10,4.0
2,C3L-00606-01,ENSG00000278267.1,MIR6859-1,chr1,17369,17436,4.0,4.0,amplified,NaN,NaN,NaN,4.0
3,C3L-00606-01,ENSG00000243485.5,MIR1302-2HG,chr1,29554,31109,4.0,4.0,amplified,0.002205,1.510508e-07,3.707889e-11,4.0
4,C3L-00606-01,ENSG00000284332.1,MIR1302-2,chr1,30366,30503,4.0,4.0,amplified,NaN,NaN,NaN,4.0


,CaseID,gene_id,gene_name,chromosome,start,end,min_copy_number,max_copy_number,status,sum_gene_expr,mean_gene_expr,variance_gene_expr,copy_number_target
0,C3L-00606-02,ENSG00000223972.5,DDX11L1,chr1,11869,14409,4.0,4.0,amplified,NaN,NaN,NaN,4.0
1,C3L-00606-02,ENSG00000227232.5,WASH7P,chr1,14404,29570,4.0,4.0,amplified,0.020324,2.268012e-06,1.261573e-09,4.0
2,C3L-00606-02,ENSG00000278267.1,MIR6859-1,chr1,17369,17436,4.0,4.0,amplified,NaN,NaN,NaN,4.0
3,C3L-00606-02,ENSG00000243485.5,MIR1302-2HG,chr1,29554,31109,4.0,4.0,amplified,0.000346,3.864082e-08,1.337829e-11,4.0
4,C3L-00606-02,ENSG00000284332.1,MIR1302-2,chr1,30366,30503,4.0,4.0,amplified,NaN,NaN,NaN,4.0


,CaseID,gene_id,gene_name,chromosome,start,end,min_copy_number,max_copy_number,status,sum_gene_expr,mean_gene_expr,variance_gene_expr,copy_number_target
0,C3L-00606-03,ENSG00000223972.5,DDX11L1,chr1,11869,14409,5.0,5.0,amplified,0.000580,3.529023e-08,2.048060e-11,5.0
1,C3L-00606-03,ENSG00000227232.5,WASH7P,chr1,14404,29570,5.0,5.0,amplified,0.048207,2.931217e-06,2.431243e-09,5.0
2,C3L-00606-03,ENSG00000278267.1,MIR6859-1,chr1,17369,17436,5.0,5.0,amplified,NaN,NaN,NaN,5.0
3,C3L-00606-03,ENSG00000243485.5,MIR1302-2HG,chr1,29554,31109,5.0,5.0,amplified,0.003089,1.878237e-07,1.627188e-10,5.0
4,C3L-00606-03,ENSG00000284332.1,MIR1302-2,chr1,30366,30503,5.0,5.0,amplified,NaN,NaN,NaN,5.0


,CaseID,gene_id,gene_name,chromosome,start,end,min_copy_number,max_copy_number,status,sum_gene_expr,mean_gene_expr,variance_gene_expr,copy_number_target
0,C3L-01287-01,ENSG00000223972.5,DDX11L1,chr1,11869,14409,4.0,4.0,amplified,NaN,NaN,NaN,4.0
1,C3L-01287-01,ENSG00000227232.5,WASH7P,chr1,14404,29570,4.0,4.0,amplified,0.056597,0.000006,4.648027e-09,4.0
2,C3L-01287-01,ENSG00000278267.1,MIR6859-1,chr1,17369,17436,4.0,4.0,amplified,NaN,NaN,NaN,4.0
3,C3L-01287-01,ENSG00000243485.5,MIR1302-2HG,chr1,29554,31109,4.0,4.0,amplified,NaN,NaN,NaN,4.0
4,C3L-01287-01,ENSG00000284332.1,MIR1302-2,chr1,30366,30503,4.0,4.0,amplified,NaN,NaN,NaN,4.0


,CaseID,gene_id,gene_name,chromosome,start,end,min_copy_number,max_copy_number,status,sum_gene_expr,mean_gene_expr,variance_gene_expr,copy_number_target
0,C3L-01287-03,ENSG00000223972.5,DDX11L1,chr1,11869,14409,1.0,1.0,deleted,NaN,NaN,NaN,1.0
1,C3L-01287-03,ENSG00000227232.5,WASH7P,chr1,14404,29570,1.0,1.0,deleted,0.011532,1.492560e-06,1.266516e-09,1.0
2,C3L-01287-03,ENSG00000278267.1,MIR6859-1,chr1,17369,17436,1.0,1.0,deleted,NaN,NaN,NaN,1.0
3,C3L-01287-03,ENSG00000243485.5,MIR1302-2HG,chr1,29554,31109,1.0,1.0,deleted,0.002492,3.225945e-07,9.807534e-11,1.0
4,C3L-01287-03,ENSG00000284332.1,MIR1302-2,chr1,30366,30503,1.0,1.0,deleted,NaN,NaN,NaN,1.0


...

,CaseID,gene_id,gene_name,chromosome,start,end,min_copy_number,max_copy_number,status,sum_gene_expr,mean_gene_expr,variance_gene_expr,copy_number_target
0,C3N-02783-05,ENSG00000223972.5,DDX11L1,chr1,11869,14409,2.0,2.0,normal,NaN,NaN,NaN,2.0
1,C3N-02783-05,ENSG00000227232.5,WASH7P,chr1,14404,29570,2.0,2.0,normal,0.011293,1.698269e-06,1.741629e-09,2.0
2,C3N-02783-05,ENSG00000278267.1,MIR6859-1,chr1,17369,17436,2.0,2.0,normal,NaN,NaN,NaN,2.0
3,C3N-02783-05,ENSG00000243485.5,MIR1302-2HG,chr1,29554,31109,2.0,2.0,normal,0.000420,6.318317e-08,2.654356e-11,2.0
4,C3N-02783-05,ENSG00000284332.1,MIR1302-2,chr1,30366,30503,2.0,2.0,normal,NaN,NaN,NaN,2.0


,CaseID,gene_id,gene_name,chromosome,start,end,min_copy_number,max_copy_number,status,sum_gene_expr,mean_gene_expr,variance_gene_expr,copy_number_target
0,C3N-02784-01-03,ENSG00000223972.5,DDX11L1,chr1,11869,14409,2.0,2.0,normal,NaN,NaN,NaN,2.0
1,C3N-02784-01-03,ENSG00000227232.5,WASH7P,chr1,14404,29570,2.0,2.0,normal,0.021425,1.190502e-06,5.022078e-10,2.0
2,C3N-02784-01-03,ENSG00000278267.1,MIR6859-1,chr1,17369,17436,2.0,2.0,normal,NaN,NaN,NaN,2.0
3,C3N-02784-01-03,ENSG00000243485.5,MIR1302-2HG,chr1,29554,31109,2.0,2.0,normal,0.000232,1.286520e-08,2.978581e-12,2.0
4,C3N-02784-01-03,ENSG00000284332.1,MIR1302-2,chr1,30366,30503,2.0,2.0,normal,NaN,NaN,NaN,2.0


,CaseID,gene_id,gene_name,chromosome,start,end,min_copy_number,max_copy_number,status,sum_gene_expr,mean_gene_expr,variance_gene_expr,copy_number_target
0,C3N-03184-02,ENSG00000223972.5,DDX11L1,chr1,11869,14409,2.0,2.0,normal,NaN,NaN,NaN,2.0
1,C3N-03184-02,ENSG00000227232.5,WASH7P,chr1,14404,29570,2.0,2.0,normal,0.01375,0.000002,9.064376e-10,2.0
2,C3N-03184-02,ENSG00000278267.1,MIR6859-1,chr1,17369,17436,2.0,2.0,normal,NaN,NaN,NaN,2.0
3,C3N-03184-02,ENSG00000243485.5,MIR1302-2HG,chr1,29554,31109,2.0,2.0,normal,NaN,NaN,NaN,2.0
4,C3N-03184-02,ENSG00000284332.1,MIR1302-2,chr1,30366,30503,2.0,2.0,normal,NaN,NaN,NaN,2.0


,CaseID,gene_id,gene_name,chromosome,start,end,min_copy_number,max_copy_number,status,sum_gene_expr,mean_gene_expr,variance_gene_expr,copy_number_target
0,C3N-03186-01,ENSG00000223972.5,DDX11L1,chr1,11869,14409,7.0,7.0,amplified,0.000911,5.538138e-08,5.043535e-11,7.0
1,C3N-03186-01,ENSG00000227232.5,WASH7P,chr1,14404,29570,7.0,7.0,amplified,0.030543,1.857297e-06,1.499790e-09,7.0
2,C3N-03186-01,ENSG00000278267.1,MIR6859-1,chr1,17369,17436,7.0,7.0,amplified,NaN,NaN,NaN,7.0
3,C3N-03186-01,ENSG00000243485.5,MIR1302-2HG,chr1,29554,31109,7.0,7.0,amplified,NaN,NaN,NaN,7.0
4,C3N-03186-01,ENSG00000284332.1,MIR1302-2,chr1,30366,30503,7.0,7.0,amplified,NaN,NaN,NaN,7.0


,CaseID,gene_id,gene_name,chromosome,start,end,min_copy_number,max_copy_number,status,sum_gene_expr,mean_gene_expr,variance_gene_expr,copy_number_target
0,C3N-03188-02,ENSG00000223972.5,DDX11L1,chr1,11869,14409,2.0,2.0,normal,NaN,NaN,NaN,2.0
1,C3N-03188-02,ENSG00000227232.5,WASH7P,chr1,14404,29570,2.0,2.0,normal,0.009492,0.000002,1.106763e-09,2.0
2,C3N-03188-02,ENSG00000278267.1,MIR6859-1,chr1,17369,17436,2.0,2.0,normal,NaN,NaN,NaN,2.0
3,C3N-03188-02,ENSG00000243485.5,MIR1302-2HG,chr1,29554,31109,2.0,2.0,normal,NaN,NaN,NaN,2.0
4,C3N-03188-02,ENSG00000284332.1,MIR1302-2,chr1,30366,30503,2.0,2.0,normal,NaN,NaN,NaN,2.0
